## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [9]:
import pandas as pd
import numpy as np
from datetime import datetime
pd.set_option('display.max_columns',50)
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import roc_curve,precision_score,auc,roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV,cross_val_score

from sklearn.svm import SVC 
def blight_model():

    
    train_data=pd.read_csv("train.csv",encoding='ISO-8859-1')
#     train_data=train_data.drop(['non_us_str_code'],axis=1)
    test_data=pd.read_csv("test.csv",encoding='ISO-8859-1')
    train_data.set_index('ticket_id',inplace=True) 
#     train_data.dropna(subset=['compliance'])
#     length=len(train_data.columns)
    total_null=train_data.isnull().sum()
    null_per=(train_data.isnull().sum()/train_data.isnull().count())
#     train_data=np.where(null_per>0.5)
    null_data=pd.concat([total_null,null_per],axis=1,keys=['total','percent'])
#     full_data=[train_data,test_data]
    train_data.drop(null_data[null_data['percent']>0.5].index,axis=1,inplace=True)
    address_data=pd.read_csv('addresses.csv')
    latlons_data=pd.read_csv('latlons.csv')
    address_lat=pd.merge(address_data,latlons_data)
    address_lat.set_index('ticket_id',inplace=True)
    train_data2=train_data.join(address_lat)
    test_data2=test_data.set_index('ticket_id').join(address_lat)
#     train_data2.drop(labels='clean_up_cost',inplace=True,axis=1)
#     test_data2.drop('clean_up_cost',inplace=True,axis=1)
    columns=[ 'violator_name','inspector_name','clean_up_cost',
       'violation_street_number', 'violation_street_name',
       'mailing_address_str_number', 'mailing_address_str_name', 'city',
       'state', 'zip_code', 'country']
    train_data2.drop(columns,inplace=True,axis=1)
    test_data2.drop(columns,inplace=True,axis=1)
    train_data2['total_payment']=train_data2['fine_amount']+train_data2['admin_fee']+train_data2['state_fee']+train_data2['late_fee']-train_data2['discount_amount']
    test_data2['total_payment']=test_data2['fine_amount']+test_data2['admin_fee']+test_data2['state_fee']+test_data2['late_fee']-test_data2['discount_amount']
#     train_data2['hearing_date']=datetime.datetime(train_data2['hearing_date'].astype(int))
#     hearing_date=datetime.strptime(train_data2['hearing_date'],'%Y-%m-%d %H:%M:%S')
#     ticket_date=datetime.strptime(train_data2['ticket_issued_date'],'%Y-%m-%d %H:%M:%S')
    
#     hearing_date1=datetime.strftime(hearing_date,'%Y-%m-%d %H:%M:%S')
#     ticket_date1=datetime.strftime(ticket_date,'%Y-%m-%d %H:%M:%S')
    def time_gap(hearing_date_str,ticket_date_str):
        if not hearing_date_str or type(hearing_date_str)!=str:return 73
        hearing_date=datetime.strptime(hearing_date_str,"%Y-%m-%d %H:%M:%S")
        ticket_date=datetime.strptime(ticket_date_str, "%Y-%m-%d %H:%M:%S")    
        diff=hearing_date-ticket_date
        return diff.days
        
    train_data2['time_gap']=train_data2.apply(lambda x:time_gap(x['hearing_date'],x['ticket_issued_date']),axis=1)
    test_data2['time_gap']=test_data2.apply(lambda x:time_gap(x['hearing_date'],x['ticket_issued_date']),axis=1)
    column_list=['hearing_date','ticket_issued_date','fine_amount','admin_fee','state_fee','late_fee','discount_amount']
    train_data2.drop(column_list,axis=1,inplace=True)
    test_data2.drop(column_list,axis=1,inplace=True)
    for i in range(len(train_data2.columns)):
        if len(train_data2[train_data2.columns[i]].unique()) < 250:
            train_data2[train_data2.columns[i]] = train_data2[train_data2.columns[i]] .astype('category')

    string_name=['agency_name','disposition']
    train_cat=pd.get_dummies(train_data2,columns=string_name)
    test_cat=pd.get_dummies(test_data2,columns=string_name)
    final_clean= [
        'balance_due',
        'compliance_detail',
        'payment_amount',
        'payment_status',
        'violation_code',
        'violation_description',
        'address']
    train_cat.drop(final_clean,axis=1,inplace=True)
#     test_cat.drop(final_clean,axis=1,inplace=True) 
    train_cat['lat'].fillna(73,inplace='true')

    train_cat['lon'].fillna(73,inplace='true')
    test_cat['lon'].fillna(73,inplace='true')
    test_cat['lat'].fillna(73,inplace='true')
    train_set=train_cat[(train_cat['compliance']==0)|(train_cat['compliance']==1)]
    list_remove=['violation_zip_code','non_us_str_code','violation_code','violation_description','grafitti_status','address']
    test_cat.drop(list_remove,axis=1,inplace=True)
    
    #training the model
    y_train=train_set['compliance']
    X_train=train_set.drop('compliance',axis=1)
    X_test=test_cat
    train_features=X_train
    train_features_set=set(train_features)
    for features in set(train_features):
        if features not in X_test:
            train_features_set.remove(features)
    train_features_list=list(train_features_set)
#     print(train_features_list)
    X_train=X_train[train_features_list]
#     return X_train
    X_test=X_test[train_features_list]
    for depth,features in zip([20,50,10],[4,12,4]):
        forest=RandomForestRegressor(n_estimators=100,max_features=features,max_depth=depth,random_state=1).fit(X_train,y_train)
        y_pred=forest.predict(X_test)
    print(roc_auc_score(y_test,y_pred))
#     print("depth:{:.1f},features:{:.1f}".format(depth,features))
    return pd.DataFrame(forest.predict(X_test),index=X_test.index,columns=['Compliance']) 
#   prinroc_auc_score(y_test,y_pred)
    
blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


NameError: name 'y_test' is not defined

In [6]:
def prediction():

#     for depth,features in zip([20,50,10],[4,12,4]):
    X_train2=X_train[train_features_list]
    X_test=X_test[train_features_list]
    forest=RandomForestRegressor(n_estimators=100,max_features=4,max_depth=20,random_state=1).fit(X_train2,y_train)
    y_pred=forest.predict(X_test)
    print(roc_auc_score(y_test,y_pred))
    print("depth:{:.1f},features:{:.1f}".format(depth,features))
    return pd.DataFrame(grid_search.predict(X_test),index=X_test.index,columns=['Compliance']) 
#   prinroc_auc_score(y_test,y_pred)
prediction()

NameError: name 'X_train' is not defined

In [7]:
def blight_model():
    import numpy as np
    import pandas as pd

    
    train_data=pd.read_csv("train.csv",encoding='ISO-8859-1')
#     train_data=train_data.drop(['non_us_str_code'],axis=1)
    test_data=pd.read_csv("test.csv",encoding='ISO-8859-1')
    features=['admin_fee', 'state_fee', 'late_fee', 'discount_amount','compliance']
    train_data=train_data[features]
    test_data=test_data[]
    return train_data
blight_model()

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,admin_fee,state_fee,late_fee,discount_amount,compliance
0,20.0,10.0,25.0,0.0,0.0
1,20.0,10.0,75.0,0.0,1.0
2,0.0,0.0,0.0,0.0,NaN
3,0.0,0.0,0.0,0.0,NaN
4,0.0,0.0,0.0,0.0,NaN
5,20.0,10.0,25.0,0.0,0.0
6,20.0,10.0,75.0,0.0,0.0
7,20.0,10.0,10.0,0.0,0.0
8,20.0,10.0,10.0,0.0,0.0
9,20.0,10.0,75.0,0.0,0.0
